<a href="https://colab.research.google.com/github/Minakshi85/MultiClass-Multilabel/blob/master/Project_1_Multiclass_Multilabel_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Multiclass Multilabel prediction For stack overflow Questions**

In [0]:
# import libraries
# Basic libraries 
import pandas as pd
import numpy as np

# regex for text cleaning 
import re

# preprocessing libraris
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import utils

# Model building libraries from keras
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Activation
from keras.layers import Input, Dense, Embedding, LSTM
from keras.models import Model
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model

# word count
from nltk import word_tokenize

#train test split
from sklearn.model_selection import train_test_split

# converting list into labels
from mlxtend.preprocessing import TransactionEncoder

In [0]:
# downloads the dataset from dropbox
! wget https://www.dropbox.com/s/5721wcs2guuykzl/stacksample.zip?dl=0

In [0]:
# unzips the contents
!unzip /content/stacksample.zip?dl=0

In [0]:
!rm /content/stacksample.zip?dl=0

In [0]:
# reads tag and question using pandas
tags = pd.read_csv(r'/content/Tags.csv')
ques = pd.read_csv(r'/content/Questions.csv', encoding='latin-1')
#ans = pd.read_csv('/content/Answers.csv', encoding='latin-1')

In [6]:
tags.shape, ques.shape 

((3750994, 2), (1264216, 7))

In [0]:
tags.head()

In [0]:
# Top 10 tags 
k = tags.Tag.value_counts().nlargest(10) #k = inner.Tag.value_counts().head(10) 
l1 = list(k.index)
l1 

In [9]:
# separates top 10 tags
tags=tags[tags['Tag'].isin(l1)]
tags.shape

(826739, 2)

In [0]:
#groupby for multilabel
tag_ques=tags.groupby('Id',as_index=False)['Tag'].agg(lambda x:x.tolist())

In [0]:
print("Shape of tags after grouping:",tag_ques.shape)
tag_ques.head()

In [12]:
# encoder for converting tags into 1 and 0
te=TransactionEncoder()
te.fit(tag_ques['Tag'])

TransactionEncoder()

In [18]:
# merge tables ques and tag_ques on 'Id'
inner = pd.merge(left=ques,right=tag_ques,how='left', on='Id')
print(inner.shape)
inner.head()

(1264216, 8)


,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,Tag
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,NaN
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,NaN
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,NaN
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,NaN
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,[c#]


In [19]:
#removes NA's
inner.dropna(inplace=True)
inner.shape

(38650, 8)

In [20]:
# merge title and body for processing
inner['content'] = inner['Title'] + inner['Body']
#drop unwanted fiels and reset table index
inner.drop(['OwnerUserId','CreationDate','ClosedDate','Score', 'Body','Title'], axis = 1, inplace =True)
inner = inner.reset_index(drop=True)
inner.head()

,Id,Tag,content
0,4080,[java],What code analysis tools do you use for your J...
1,7990,[c#],Printing from a .NET Service<p>I am working on...
2,15880,[c#],Read from .msg files<p>I need to read from Out...
3,16140,[java],What's the best way to get started with OSGI?<...
4,23250,[c#],"When do you use the ""this"" keyword?<p>I was cu..."


Preprocessing of text by removing html tags single 

In [0]:
# removing HTML tags 
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [0]:
X_new = []
for i in inner['content']:
   X_new.append(remove_tags(i))

In [0]:
inner['refined_text'] = X_new

**remove special characters, numbers, punctuations**

In [24]:
inner['refined_text'] = inner['refined_text'].str.replace("[^a-zA-Z]", " ")
inner.head()

,Id,Tag,content,refined_text
0,4080,[java],What code analysis tools do you use for your J...,What code analysis tools do you use for your J...
1,7990,[c#],Printing from a .NET Service<p>I am working on...,Printing from a NET ServiceI am working on a ...
2,15880,[c#],Read from .msg files<p>I need to read from Out...,Read from msg filesI need to read from Outloo...
3,16140,[java],What's the best way to get started with OSGI?<...,What s the best way to get started with OSGI W...
4,23250,[c#],"When do you use the ""this"" keyword?<p>I was cu...",When do you use the this keyword I was curio...


In [25]:
X = inner['refined_text']
# converts labels into binary encoded form
Y = te.transform(inner['Tag']).astype(int)
print("Shape of X and Y is:", X.shape, Y.shape)
X.head()

Shape of X and Y is: (38650,) (38650, 10)


0    What code analysis tools do you use for your J...
1    Printing from a  NET ServiceI am working on a ...
2    Read from  msg filesI need to read from Outloo...
3    What s the best way to get started with OSGI W...
4    When do you use the  this  keyword I was curio...
Name: refined_text, dtype: object

In [26]:
Y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 0]])

In [27]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((30920,), (30920, 10), (7730,), (7730, 10))

In [28]:
x_train.head()

1777     How do I detect IE   using JQuery    Possible ...
30714    Evaluate expression from a textboxHow to evalu...
4924     Motivation for mutable in C     Possible Dupli...
878      Visual Studio  Lost CDI have my original CD ho...
23555    Cannot find symbol run  I have a bit knowledge...
Name: refined_text, dtype: object

In [29]:
import nltk
nltk.download('punkt')  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [30]:
# counting word counts in train
sent_lens=[len(word_tokenize(x)) for x in x_train]
print(max(sent_lens))

4592


In [31]:
print("99 percentile :", np.percentile(sent_lens,99))
print("95 percentile :", np.percentile(sent_lens,95))
print("90 percentile: ", np.percentile(sent_lens,90))

99 percentile : 882.6200000000026
95 percentile : 399.0
90 percentile:  278.0


In [0]:
max_len = 400

**Tokenazation Process**

In [0]:
tok = Tokenizer(lower=True, char_level = False, split=' ')
tok.fit_on_texts(x_train)
# conversion into sequence
seq_train = tok.texts_to_sequences(x_train)
seq_test = tok.texts_to_sequences(x_test)

In [0]:
# padding
seq_train_matrix = pad_sequences(seq_train,maxlen = max_len)
seq_test_matrix = pad_sequences(seq_test,maxlen = max_len)

In [35]:
vocab_size=len(tok.word_index)
vocab_size

111615

In [0]:
seq_train_matrix[2000]

**Parameters setting**

In [0]:
# set parameters:
#vocab_size
batch_size = 128
embedding_dims = 50
filters = 250
hidden_dims = 250
kernel_size = 3
epochs = 100

**Model Building**

In [0]:
model = Sequential()
model.add(Embedding(vocab_size+1,
                    embedding_dims,
                    input_length = max_len))
model.add(Dropout(0.2))
model.add(Conv1D(filters,
                 kernel_size,
                 padding = 'valid',
                 activation = 'relu',
                 strides = 1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(256))
model.add(Dropout(0.2))
model.add(Activation('tanh'))

model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(10))
model.add(Activation('sigmoid'))

In [71]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 400, 50)           5580800   
_________________________________________________________________
dropout_4 (Dropout)          (None, 400, 50)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 398, 250)          37750     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 250)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               64256     
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
activation_4 (Activation)    (None, 256)              

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
cp = ModelCheckpoint('/content/best_model.h5', 
                             monitor='val_loss',
                             verbose=1, 
                             save_best_only=True)

In [0]:
es = EarlyStopping(monitor='val_loss', 
                          patience=7,
                          verbose=1)

In [75]:
model.fit(seq_train_matrix, y_train,
          batch_size=batch_size,
          epochs=50,
          validation_data=(seq_test_matrix, y_test),
          callbacks = [es,cp])

Train on 30920 samples, validate on 7730 samples
Epoch 1/50
30920/30920 [==============================] - 4s 125us/step - loss: 0.2805 - acc: 0.9034 - val_loss: 0.1443 - val_acc: 0.9456

Epoch 00001: val_loss improved from inf to 0.14428, saving model to /content/best_model.h5
Epoch 2/50
30920/30920 [==============================] - 3s 102us/step - loss: 0.1213 - acc: 0.9544 - val_loss: 0.1005 - val_acc: 0.9616

Epoch 00002: val_loss improved from 0.14428 to 0.10048, saving model to /content/best_model.h5
Epoch 3/50
30920/30920 [==============================] - 3s 102us/step - loss: 0.0880 - acc: 0.9671 - val_loss: 0.0954 - val_acc: 0.9636

Epoch 00003: val_loss improved from 0.10048 to 0.09537, saving model to /content/best_model.h5
Epoch 4/50
30920/30920 [==============================] - 3s 102us/step - loss: 0.0685 - acc: 0.9746 - val_loss: 0.1004 - val_acc: 0.9629

Epoch 00004: val_loss did not improve from 0.09537
Epoch 5/50
30920/30920 [==============================] - 3s 10

In [0]:
saved_model = load_model('/content/best_model.h5')

In [0]:
y_predicted = saved_model.predict(seq_test_matrix)

In [0]:
y_label = np.where(y_predicted>0.5, 1,0)

In [94]:
y_label[2389]

array([0, 0, 0, 0, 0, 0, 1, 1, 0, 0])

In [95]:
y_test[2389]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0])

In [0]:
from sklearn.metrics import accuracy_score

In [97]:
print("Accuracy Score :", (accuracy_score(y_test, y_label))*100,"%")

Accuracy Score : 74.24320827943079 %
